# Cartooning of an Image 

## Libraries to be used

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from skimage import data, io, filters
from matplotlib.pyplot import imshow, show, subplot, title, get_cmap
from skimage import feature
from skimage.restoration import denoise_bilateral as bilateralfilter
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage.transform import rescale, resize
from skimage.util import img_as_ubyte


## Edge Detection

### Convert RGB to Gray Scale

In [ ]:
def RGBtoGray (img):
    newImage=np.zeros((img.shape[0], img.shape[1]))
    for i in range (img.shape[0]):
        for j in range (img.shape[1]):
            newImage[i][j]= int((img[i][j][0]/3)+ (img[i][j][1]/3)+ (img[i][j][2]/3))
    return newImage
fruit = data.astronaut()
grayFruit = RGBtoGray(fruit)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(fruit ,cmap=plt.cm.gray)
ax2.imshow(grayFruit ,cmap=plt.cm.gray)
plt.show()

### Image Filtering  Using Adaptive Median Filter

In [ ]:
def adaptive_median_filter(img,s,start_size):
    new_image= img
    
    
    for i in range(1,img.shape[0]-(math.floor(s/2))):
        for j in range(1,img.shape[1]-(math.floor(s/2))):
            start = start_size
            changed = True
            while (s>start):
                kernel= np.ones((start,start))
                
                new_image[i][j] = med_calc(img, kernel, i, j)
                
                if (new_image[i][j]==-1):
                    changed = False
                    start+=1
                else:
                    changed = True
                    start=s
            
            if not changed:
                new_image[i][j] = img[i][j]
    return new_image
       
        
def med_calc(img,kernel, i, j):
    
    klist=[]
    m= math.floor((len(kernel))/2)
    n= len(kernel)-m
    
    for x in range(-m,n-1):
        for y in range(-m,n-1):
            klist.append(img[i+x][j+y])
            
    klist.sort()
    med =0
    
    if (len(klist) % 2)==0:
        med =math.floor(((klist[math.floor(len(klist)/2)]) + (klist[math.floor((len(klist)/2))+1]))/2)
    else:
        med=math.floor(klist[math.ceil(len(klist)/2)])
        
    a1=med-klist[0]
    a2=med-klist[len(klist)-1]
    res=0
    
    if (a1>0 and a2<0):
        b1=img[i][j]-klist[0]
        b2=img[i][j]-klist[len(klist)-1]
        if(b1>0 and b2<0):
            res=img[i][j]
        else: 
            res= med
    else:
        res=-1
    return res


original = data.astronaut()
gray = RGBtoGray(original)
filtered = adaptive_median_filter(gray,9,5)


fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(gray ,cmap=plt.cm.gray)
ax2.imshow(filtered ,cmap=plt.cm.gray)
plt.show()

### Difference of Gaussian Edge Detection

In [ ]:
def gaussian_filter(img,k_size,sigma=1):
    
    new_image=np.zeros(img.shape)
    kernel = gaussian_kernel (k_size, sigma)
    for i in range(1,img.shape[0]-(math.floor(k_size/2))):
        for j in range(1,img.shape[1]-(math.floor(k_size/2))):
            new_val = conv_mult(img, kernel, i, j)
            if new_val>255:
              new_image[i][j] = 255
            else:
              new_image[i][j]= new_val
    return new_image

def gaussian_kernel (k_size, sigma=1):
    kernel_half = math.floor (k_size/2)
    x, y= np.mgrid[-kernel_half:kernel_half+1, -kernel_half:kernel_half+1]
    kernel= (1/(2.0 * np.pi * sigma**2)) * (np.exp(-((x**2 + y**2)/(2.0*sigma**2))))
    return kernel


def conv_mult(img,kernel, i, j):
    result = 0;
    m= math.floor((len(kernel))/2)
    n= len(kernel)-m
    for x in range(-m,n):
        for y in range(-m,n):
            result += img[i+x][j+y] * kernel[x][y]
    return result

def subtract(img1,img2):
    new_img = np.zeros(img1.shape, dtype=np.uint8)
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            new_img[i][j] = abs(int(img1[i][j])-int(img2[i][j]))
    return new_img

def DoG(img, size1, size2):
    gaussian1 = gaussian_filter(img,size1,sigma=0.75)
    gaussian2 = gaussian_filter(img,size1,sigma=1.25)
    diff = subtract(gaussian1,gaussian2)
    return diff


original = data.astronaut()
gray = RGBtoGray(original)
edge_dog = DoG(gray, 7, 9)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(original ,cmap=plt.cm.gray)
ax2.imshow(edge_dog ,cmap=plt.cm.gray)
plt.show()


#### Image Thresholding 

In [ ]:
def binary_inverted (img, thr_value):
    new_image = np.zeros(img.shape)
    for i in range (img.shape[0]):
        for j in range (img.shape[1]):
            if (img[i][j]> thr_value):
                new_image[i][j]=0
            else:
                new_image[i][j]=255
    return new_image

inverted_edges_dog = binary_inverted(edge_dog,9)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(edge_dog ,cmap=plt.cm.gray)
ax2.imshow(inverted_edges_dog ,cmap=plt.cm.gray)
plt.show()

In [ ]:
car = io.imread('/content/car.png')
baby = io.imread('/content/baby.png')
lenna = io.imread('/content/Lenna.png')
salmeen = io.imread('/content/salmeen.png')


gray1 = RGBtoGray(car)
gray2 = RGBtoGray(baby)
gray3 = RGBtoGray(lenna)
gray4 = RGBtoGray(salmeen)


edge_dog1 = DoG(gray1, 7, 9)
edge_dog2 = DoG(gray2, 7, 9)
edge_dog3 = DoG(gray3, 7, 9)
edge_dog4 = DoG(gray4, 7, 9)


inverted_edges_dog1 = binary_inverted(edge_dog1,5)
inverted_edges_dog2 = binary_inverted(edge_dog2,5)
inverted_edges_dog3 = binary_inverted(edge_dog3,5)
inverted_edges_dog4 = binary_inverted(edge_dog4,5)
i

In [ ]:
boy = io.imread('/content/boy.PNG')
gray5= RGBtoGray(boy)
edge_dog5 = DoG(gray5, 7, 9)
nverted_edges_dog5 = binary_inverted(edge_dog5,5)


### Sobel Edge Detection 

In [ ]:
def Sobel_v(img):
    new_image = np.zeros(img.shape)
    for i in range(1,img.shape[0]-1):
        for j in range(1,img.shape[1]-1):
            new_image[i][j] = conv_mult1(img, [[-1,0,1],[-2,0,2],[-1,0,1]], i, j)
    
    return new_image

def Sobel_h(img):
    new_image = np.zeros(img.shape)
    for i in range(1,img.shape[0]-1):
        for j in range(1,img.shape[1]-1):
            new_image[i][j] = conv_mult1(img, [[-1,-2,-1],[0,0,0],[1,2,1]], i, j)
    
    return new_image

def sobel_sum(img):
    res= Sobel_h(img)+ Sobel_v(img)
    return res

def conv_mult1(img,kernel, i, j):
    result = 0;
    for x in range(-1,2):
        for y in range(-1,2):
            result += img[i+x][j+y] * kernel[x][y];
    return result

original = data.astronaut()
gray = RGBtoGray(original)
edge_sobel = sobel_sum (gray)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(original ,cmap=plt.cm.gray)
ax2.imshow(edge_sobel ,cmap=plt.cm.gray)
plt.show()


In [ ]:
inverted_edges_sobel = binary_inverted(edge_sobel, 50)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(edge_sobel ,cmap=plt.cm.gray)
ax2.imshow(inverted_edges_sobel ,cmap=plt.cm.gray)
plt.show()

### Laplace Edge Detection

In [ ]:
def Laplace(img):
    new_image = np.zeros(img.shape)
    for i in range(1,img.shape[0]-1):
        for j in range(1,img.shape[1]-1):
            new_image[i][j] = conv_mult(img,[[0,1,0],[1,-4,1],[0,1,0]],i,j)
    return new_image

def conv_mult(img,kernel, i, j):
    result = 0;
    for x in range(-1,2):
        for y in range(-1,2):
            result += img[i+x][j+y] * kernel[x][y];
    return result

original = data.astronaut()
gray = RGBtoGray(original)
edge_laplace = Laplace (gray)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(original ,cmap=plt.cm.gray)
ax2.imshow(edge_laplace ,cmap=plt.cm.gray)
plt.show()


In [ ]:
inverted_edges_laplace = binary_inverted(edge_laplace,27)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(edge_laplace ,cmap=plt.cm.gray)
ax2.imshow(inverted_edges_laplace ,cmap=plt.cm.gray)
plt.show()

### Canny Edge Detection 

In [ ]:
def mean_calc(img):
    sumi = 0
    counter = 0
    for i in range (img.shape[0]):
        for j in range (img.shape[1]):
            sumi += img[i][j]
            counter +=1
    mean = int(sumi/counter)
    return mean
def canny_edge(img):
    mean= mean_calc(img)
    low= 0.001* mean
    high= 0.5* mean
    new_image = feature.canny(img, sigma=3,low_threshold=low,high_threshold=high)
    return new_image

original = data.astronaut()
gray = RGBtoGray(original)
edge_canny = canny_edge (gray)

fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(original ,cmap=plt.cm.gray)
ax2.imshow(edge_canny ,cmap=plt.cm.gray)
plt.show()


In [ ]:
inverted_edges_canny = binary_inverted(edge_canny, 0)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(edge_canny ,cmap=plt.cm.gray)
ax2.imshow(inverted_edges_canny ,cmap=plt.cm.gray)
plt.show()

## Region Smothing 

### Bilateral Filtering 

In [ ]:
def bilateral_multiple(img, iterations,sigma_spatial, sigma_range):
    new_image = img
    for i in range (0,iterations):
        new_image= bilateralfilter(new_image, win_size=max(5, 2*math.ceil(3*sigma_spatial)+1),
                               sigma_color=sigma_range,sigma_spatial=sigma_spatial, bins=100000000, multichannel=True)
        
    return img_as_ubyte(new_image)
fruit = data.astronaut()
filtered2= bilateral_multiple(fruit,1,3,20)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(fruit ,cmap=plt.cm.gray)
ax2.imshow(filtered2 ,cmap=plt.cm.gray)
plt.show()

##**Combining the Filtered Image with Edge Detection Image**

In [ ]:
def combine(img_original,edges_inverted):
    cartoon = np.zeros(img_original.shape)

    for i in range(img_original.shape[0]):
        for j in range(img_original.shape[1]):
            if edges_inverted[i][j] != 0:
                cartoon[i][j][0] = img_original[i][j][0]
                cartoon[i][j][1] = img_original[i][j][1]
                cartoon[i][j][2] = img_original[i][j][2]

    return cartoon.astype(int)


##Image Resizing


In [ ]:
def resize_img(img, factor):
    length, width, channels = img.shape
    new_image = resize(img,(int(length/ factor), int(width/ factor)))
    return new_image
                       
fruit = data.astronaut()
filtered1= resize_img(fruit, 2)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(filtered1 ,cmap=plt.cm.gray)
ax2.imshow(fruit ,cmap=plt.cm.gray)
plt.show()

In [ ]:
fruit = data.astronaut()
filtered1= resize_img(fruit, 0.5)
filtered2= resize_img(filtered1,2)
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(16, 8)
ax1.imshow(filtered1 ,cmap=plt.cm.gray)
ax2.imshow(filtered2 ,cmap=plt.cm.gray)
plt.show()

##Main Function to Convert Real Images to Cartoon

In [ ]:
#must run all previous cells in order
def cartoon_image(img, edges="laplace", iterations=1, sigma_spatial=3, sigma_range=10):
    
    gray_image = RGBtoGray (img).astype(int)
    denoised_image = adaptive_median_filter(gray_image,21,9)
    edges_img = None
    if edges == "laplace":
        Laplace_edge = Laplace (denoised_image)
        edges_img = binary_inverted(Laplace_edge,50)
    if edges == "sobel":
        sobel_edge = sobel_sum (denoised_image)
        edges_img = binary_inverted(sobel_edge,50)
    if edges == "canny":
        canny_img = canny_edge (denoised_image)
        edges_img = binary_inverted(canny_img,0)
    if edges == "dog":
        edges_img = np.ones(gray_image.shape)


    # small_image = resize_img(img,1.05)
    filtered_bilateral = bilateral_multiple(img, iterations,sigma_spatial, sigma_range)
    # fullsize_image = resize_img(filtered_bilateral,0.95238)
    result = combine(filtered_bilateral, edges_img)
    return result





###Test Case 1

In [ ]:
astronaut = data.astronaut()
cartoon1 = cartoon_image(astronaut, edges="laplace", iterations=1, sigma_spatial=3, sigma_range=20)
cartoon2 = cartoon_image(astronaut, edges="sobel", iterations=1, sigma_spatial=3, sigma_range=20)
cartoon3 = cartoon_image(astronaut, edges="canny", iterations=1, sigma_spatial=3, sigma_range=20)
cartoon4 = cartoon_image(astronaut, edges="dog", iterations=1, sigma_spatial=3, sigma_range=20)

 
fig, ((ax1,ax2) , (ax3,ax4)) = plt.subplots(2,2)
fig.set_size_inches(20, 20)
ax1.imshow(cartoon1)
ax2.imshow(cartoon2)
ax3.imshow(cartoon3)
ax4.imshow(combine(cartoon4,inverted_edges_dog))

ax1.set_title("laplace")
ax2.set_title("sobel")
ax3.set_title("canny")
ax4.set_title("dog")
plt.show()

###Test Case 2


In [ ]:
car = io.imread('/content/car.png')
cartoon5 = cartoon_image(car, edges="laplace", iterations=1, sigma_spatial=6, sigma_range=10)
cartoon6 = cartoon_image(car, edges="sobel", iterations=1, sigma_spatial=6, sigma_range=10)
cartoon7 = cartoon_image(car, edges="canny", iterations=1, sigma_spatial=6, sigma_range=10)
cartoon8 = cartoon_image(car, edges="dog", iterations=1, sigma_spatial=6, sigma_range=10)


fig, ((ax1,ax2) , (ax3,ax4)) = plt.subplots(2,2)
fig.set_size_inches(20, 20)
ax1.imshow(cartoon5)
ax2.imshow(cartoon6)
ax3.imshow(cartoon7)
ax4.imshow(combine(cartoon8,inverted_edges_dog1))

ax1.set_title("laplace")
ax2.set_title("sobel")
ax3.set_title("canny")
ax4.set_title("dog" )


plt.show()

###Test Case 3

In [ ]:
baby = io.imread('/content/baby.png')
cartoon9 = cartoon_image(baby, edges="laplace", iterations=1, sigma_spatial=5, sigma_range=12)
cartoon10 = cartoon_image(baby, edges="sobel", iterations=1, sigma_spatial=5, sigma_range=12)
cartoon11 = cartoon_image(baby, edges="canny", iterations=1, sigma_spatial=5, sigma_range=12)
cartoon12 = cartoon_image(baby, edges="dog", iterations=1, sigma_spatial=5, sigma_range=12)


fig, ((ax1,ax2) , (ax3,ax4)) = plt.subplots(2,2)
fig.set_size_inches(20, 20)
ax1.imshow(cartoon9)
ax2.imshow(cartoon10)
ax3.imshow(cartoon11)
ax4.imshow(combine(cartoon12,inverted_edges_dog2))

ax1.set_title("laplace")
ax2.set_title("sobel")
ax3.set_title("canny")
ax4.set_title("dog" )


plt.show()

###Test Case 4

In [ ]:
lenna = io.imread('/content/Lenna.png')
cartoon13 = cartoon_image(lenna, edges="laplace", iterations=1, sigma_spatial=6, sigma_range=10)
cartoon14 = cartoon_image(lenna, edges="sobel", iterations=1, sigma_spatial=6, sigma_range=10)
cartoon15 = cartoon_image(lenna, edges="canny", iterations=1, sigma_spatial=6, sigma_range=10)
cartoon16 = cartoon_image(lenna, edges="dog", iterations=1, sigma_spatial=6, sigma_range=10)


fig, ((ax1,ax2) , (ax3,ax4)) = plt.subplots(2,2)
fig.set_size_inches(20, 20)
ax1.imshow(cartoon13)
ax2.imshow(cartoon14)
ax3.imshow(cartoon15)
ax4.imshow(combine(cartoon16,inverted_edges_dog3))

ax1.set_title("laplace")
ax2.set_title("sobel")
ax3.set_title("canny")
ax4.set_title("dog" )


plt.show()